In [1]:
from astroquery.mast import Observations
from ccdproc import ImageFileCollection
from astropy.table import Table
from astropy.io import fits
from astropy.io import ascii
from astropy.visualization import ZScaleInterval
from IPython.display import Image
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
from drizzlepac import tweakreg
from drizzlepac import astrodrizzle



The following task in the stsci.skypac package can be run with TEAL:
                                    skymatch                                    
The following tasks in the drizzlepac package can be run with TEAL:
    astrodrizzle       config_testbed      imagefindpars           mapreg       
       photeq            pixreplace           pixtopix            pixtosky      
  refimagefindpars       resetbits          runastrodriz          skytopix      
     tweakback            tweakreg           updatenpol


In [55]:
science_list = Observations.query_criteria(proposal_id='16231', filters='F150LP',obs_id='jec409*')
Observations.download_products(science_list['obsid'], mrp_only=False, download_dir='./science',\
                               productSubGroupDescription=['FLT'])

science_files = glob.glob(os.path.join(os.curdir, 'science', 'mastDownload', 'HST', '*', '*fits'))
for im in science_files:
    root = im.split('/')[-1]
    os.rename(im, './' + root)
shutil.rmtree('science/')

In [3]:
science_list = Observations.query_criteria(proposal_id='16231', filters='F150LP',obs_id='jec403*')
Observations.download_products(science_list['obsid'], mrp_only=False, download_dir='./science',\
                               productSubGroupDescription=['FLT'])

science_files = glob.glob(os.path.join(os.curdir, 'science', 'mastDownload', 'HST', '*', '*fits'))
for im in science_files:
    root = im.split('/')[-1]
    os.rename(im, './' + root)
shutil.rmtree('science/')

In [4]:
no = 'hst_16231_01_acs_sbc_f150lp_jec401no_flt.fits'
nr = 'hst_16231_01_acs_sbc_f150lp_jec401nr_flt.fits'
nv = 'hst_16231_01_acs_sbc_f150lp_jec401nv_flt.fits'
nx = 'hst_16231_01_acs_sbc_f150lp_jec401nx_flt.fits'
lst_1 = [no, nr, nv, nx]

In [5]:
ib = 'hst_16231_03_acs_sbc_f150lp_jec403ib_flt.fits'
ic = 'hst_16231_03_acs_sbc_f150lp_jec403ic_flt.fits'
id3 = 'hst_16231_03_acs_sbc_f150lp_jec403id_flt.fits'
ie = 'hst_16231_03_acs_sbc_f150lp_jec403ie_flt.fits'
lst_3 = [ib, ic, id3, ie]

In [ ]:
no_f = fits.open(no)
data = no_f[1].data

In [ ]:
plt.imshow(data)

In [ ]:
plt.imshow(data[410:430, 480:500])

In [ ]:
plt.imshow(data[390:450, 470:520])

In [ ]:
data_small = no_f[1].data[410:430, 480:500]

In [ ]:
max_i = 0
max_j = 0
maxval = 0
for i in range(len(data_small)):
    for j in range(len(data_small[i])):
        if data_small[i][j] > maxval:
            maxval = data_small[i][j]
            max_i = i
            max_j = j
            
print(max_i, max_j, maxval)

In [ ]:
data[417][489]

In [ ]:
data[510][]

In [ ]:
CRVAL1 = no_f[1].header['CRVAL1']
CRVAL2 = no_f[1].header['CRVAL2']
CRPIX1 = no_f[1].header['CRPIX1']
CRPIX2 = no_f[1].header['CRPIX2']
CD1_1 = no_f[1].header['CD1_1']
CD1_2 = no_f[1].header['CD1_2']
CD2_1 = no_f[1].header['CD2_1']
CD2_2 = no_f[1].header['CD2_2']

In [ ]:
A = np.array([[CD1_1, CD1_2], [CD2_1, CD2_2]])
A_inv = np.linalg.inv(A)
B = np.array([[CRVAL1],[CRVAL2]])
r0 = np.array([[CRPIX1],[CRPIX2]])

In [ ]:
B

In [ ]:
x, y = 512, 512
r = np.array([[x],[y]])

In [ ]:
def coo_pixtosky(r):
    return np.matmul(A, r-r0) + B

In [ ]:
coo_pixtosky(np,ar)

In [ ]:
def coo_skytopix(r_sky):
    return np.matmul(A_inv, r_sky - B) + r0

In [ ]:
RA_TARG = no_f[0].header['RA_TARG'] 
DEC_TARG = no_f[0].header['DEC_TARG']
r_sky_TARG = np.array([[RA_TARG], [DEC_TARG]])

In [ ]:
X_TARG, Y_TARG = int(coo_skytopix(r_sky_TARG)[0]), int(coo_skytopix(r_sky_TARG)[1])

In [ ]:
s = 50
max_i = 0
max_j = 0
maxval = 0
for i in range(Y_TARG-s, Y_TARG+s+1):
    for j in range(X_TARG-s, X_TARG+s+1):
        if data[i][j] >maxval:
            maxval = data[i][j]
            max_i = i
            max_j = j 
print(maxval, max_i, max_j)

In [ ]:
data[417][489]

In [6]:
def find_center(inputfilename):
    file = fits.open(inputfilename)
    #WCS info from header
    CRVAL1 = file[1].header['CRVAL1']
    CRVAL2 = file[1].header['CRVAL2']
    CRPIX1 = file[1].header['CRPIX1']
    CRPIX2 = file[1].header['CRPIX2']
    CD1_1 = file[1].header['CD1_1']
    CD1_2 = file[1].header['CD1_2']
    CD2_1 = file[1].header['CD2_1']
    CD2_2 = file[1].header['CD2_2']
    
    A = np.array([[CD1_1, CD1_2], [CD2_1, CD2_2]])
    A_inv = np.linalg.inv(A)
    B = np.array([[CRVAL1],[CRVAL2]])
    r0 = np.array([[CRPIX1],[CRPIX2]])
    
    def coo_pixtosky(r):
        return np.matmul(A, r-r0) + B
    
    def coo_skytopix(r_sky):
        return np.matmul(A_inv, r_sky - B) + r0
    
    #Target location info
    RA_TARG = file[0].header['RA_TARG'] 
    DEC_TARG = file[0].header['DEC_TARG']
    r_sky_TARG = np.array([[RA_TARG], [DEC_TARG]])
    X_TARG, Y_TARG = int(coo_skytopix(r_sky_TARG)[0]), int(coo_skytopix(r_sky_TARG)[1])
    
    data = file[1].data
    
    s = 100
    max_i = 0
    max_j = 0
    maxval = 0
    for i in range(Y_TARG-s, Y_TARG+s+1):
        for j in range(X_TARG-s, X_TARG+s+1):
            if data[i][j] > maxval:
                maxval = data[i][j]
                max_i = i
                max_j = j 
    r = 50
    total_flux = 0
    i_integral = 0
    j_integral = 0
    for i in range(max_i-r, max_i+r+1):
        for j in range(max_j-r, max_j+r+1):
            total_flux += data[i][j]
            i_integral += i*data[i][j]
            j_integral += j*data[i][j]
    
    i_center = i_integral/total_flux
    j_center = j_integral/total_flux
    
    X_center = j_center +1
    Y_center = i_center +1
    r_center = np.array([[X_center], [Y_center]])
    
    #print('X_center =', X_center)
    #print('Y_center =', Y_center)
    sky_coo_center = coo_pixtosky(r_center)
    #RA_center, DEC_center = sky_coo_center[0][0], sky_coo_center[1][0]
    return r_center, sky_coo_center


In [7]:
for file in lst_1+lst_3:
    
    print(fits.open(file)[0].header['ROOTNAME'][-4:])
    print(find_center(file))

1noq
(array([[487.54574343],
       [419.6677651 ]]), array([[3.19600544e+02],
       [2.91488262e-01]]))
1nrq
(array([[512.79115843],
       [427.91858324]]), array([[3.19600547e+02],
       [2.91493212e-01]]))
1nvq
(array([[500.78426636],
       [444.29299893]]), array([[3.19600555e+02],
       [2.91503937e-01]]))
1nxq
(array([[477.04900291],
       [436.56076792]]), array([[3.19600548e+02],
       [2.91513909e-01]]))
3ibq
(array([[536.01293793],
       [508.67812468]]), array([[3.19600559e+02],
       [2.91539057e-01]]))
3icq
(array([[561.42728718],
       [515.43368142]]), array([[3.19600552e+02],
       [2.91537808e-01]]))
3idq
(array([[549.57123242],
       [529.92772526]]), array([[3.19600545e+02],
       [2.91540508e-01]]))
3ieq
(array([[524.75250787],
       [522.15504653]]), array([[3.19600542e+02],
       [2.91541044e-01]]))


In [23]:
def avg_center(lst):
    a = np.array([[0.0],[0.0]])
    for item in lst:
        a += find_center(item)[1]
    return a/len(lst)

In [24]:
avg_center0 = avg_center(lst_1 + lst_3)
avg_center0

array([[3.19600549e+02],
       [2.91519717e-01]])

In [27]:
avg_center1 = avg_center(lst_1)
avg_center1

array([[3.19600549e+02],
       [2.91499830e-01]])

In [42]:
def new_CRVALs(inputfilename, ref_center):
    file = fits.open(inputfilename)
    #WCS info from header
    CRVAL1 = file[1].header['CRVAL1']
    CRVAL2 = file[1].header['CRVAL2']
    CRPIX1 = file[1].header['CRPIX1']
    CRPIX2 = file[1].header['CRPIX2']
    CD1_1 = file[1].header['CD1_1']
    CD1_2 = file[1].header['CD1_2']
    CD2_1 = file[1].header['CD2_1']
    CD2_2 = file[1].header['CD2_2']
    
    A = np.array([[CD1_1, CD1_2], [CD2_1, CD2_2]])
    r0 = np.array([[CRPIX1],[CRPIX2]])
    
    coo_center = find_center(inputfilename)
    r_center = coo_center[0]
    sky_coo_center = coo_center[1]
    
    new_CRVALs = ref_center - np.matmul(A, r_center - r0)
    return new_CRVALs

In [43]:
for file in lst_1+lst_3:
    
    print(fits.open(file)[0].header['ROOTNAME'][-4:])
    print(new_CRVALs(file, avg_center0)[0])

1noq
[319.60108926]
1nrq
[319.60114088]
1nvq
[319.60097426]
1nxq
[319.60092554]
3ibq
[319.60069479]
3icq
[319.60078249]
3idq
[319.60062715]
3ieq
[319.60054913]


In [44]:
for item in lst_1+lst_3:
    file = fits.open(item, mode = 'update')
    new_crvals = new_CRVALs(item, avg_center0)
    for i in [1,2,3]:
        file[i].header['CRVAL1'] = new_crvals[0][0]
        file[i].header['CRVAL2'] = new_crvals[1][0]
    file.flush()

In [19]:
file = fits.open(nr)
file[1].header['CRVAL1']

319.60113936155

In [ ]:
file = fits.open(nr)

In [46]:
astrodrizzle.AstroDrizzle(lst_1 + lst_3, 
                              output = 'total_visit1+3_test4', 
                              clean = True,
                              build = True,
                              context = True,
                              runfile = 'total_visit1+3_drz.log',
                              num_cores = 1,
                              in_memory = False,
                              resetbits = 4096,
                              median = False,
                              static = False,
                              skysub = False,
                              driz_separate = False,
                              blot = False,
                              driz_cr = False,
                              final_bits = 0,
                              final_outnx=1947,
                              final_outny=1960, 
                              final_rot = 0.0,
                              final_scale = 0.025,
                              final_wcs = True)

INFO:drizzlepac.util:Setting up logfile :  total_visit1+3_drz.log


Setting up logfile :  total_visit1+3_drz.log


INFO:drizzlepac.astrodrizzle:AstroDrizzle log file: total_visit1+3_drz.log


AstroDrizzle log file: total_visit1+3_drz.log


INFO:drizzlepac.astrodrizzle:AstroDrizzle Version 3.4.2 started at: 20:45:05.470 (12/06/2022)


...

INFO:drizzlepac.util:                  Total          4.4012 sec.


                  Total          4.4012 sec.


INFO:drizzlepac.util:


INFO:drizzlepac.imageObject:Removing intermediate files for hst_16231_01_acs_sbc_f150lp_jec401no_flt.fits
INFO:drizzlepac.imageObject:Removing intermediate files for hst_16231_01_acs_sbc_f150lp_jec401nr_flt.fits
INFO:drizzlepac.imageObject:Removing intermediate files for hst_16231_01_acs_sbc_f150lp_jec401nv_flt.fits
INFO:drizzlepac.imageObject:Removing intermediate files for hst_16231_01_acs_sbc_f150lp_jec401nx_flt.fits
INFO:drizzlepac.imageObject:Removing intermediate files for hst_16231_03_acs_sbc_f150lp_jec403ib_flt.fits
INFO:drizzlepac.imageObject:Removing intermediate files for hst_16231_03_acs_sbc_f150lp_jec403ic_flt.fits
INFO:drizzlepac.imageObject:Removing intermediate files for hst_16231_03_acs_sbc_f150lp_jec403id_flt.fits
INFO:drizzlepac.imageObject:Removing intermediate files for hst_16231_03_acs_sbc_f150lp_jec403ie_flt.fits
INFO:drizzlepac.util:Trailer file written to:  total_visit1+3_drz.log


Trailer file written to:  total_visit1+3_drz.log


In [45]:
collect = ImageFileCollection('./', filenames = lst_1 + lst_3, ext=2, glob_include = '*flt.fits',
                                 keywords=["CRVAL1", "CRVAL2", "CD1_1", "CD1_2",
                                           "CD2_1", "CD2_2"])
table = collect.summary
table['CRVAL1'].format = '.6f'
table['CRVAL2'].format = '.6f'
table['CD1_1'].format = '.11f'
table['CD1_2'].format = '.11f'
table['CD2_1'].format = '.11f'
table['CD2_2'].format = '.11f'
table

file,CRVAL1,CRVAL2,CD1_1,CD1_2,CD2_1,CD2_2
str45,float64,float64,float64,float64,float64,float64
hst_16231_01_acs_sbc_f150lp_jec401no_flt.fits,319.601089,0.292144,-0.00000433217,0.00000699882,0.00000830078,0.00000456484
hst_16231_01_acs_sbc_f150lp_jec401nr_flt.fits,319.601141,0.291897,-0.00000433212,0.00000699873,0.00000830067,0.00000456478
hst_16231_01_acs_sbc_f150lp_jec401nv_flt.fits,319.600974,0.291922,-0.00000433204,0.00000699861,0.00000830054,0.00000456470
hst_16231_01_acs_sbc_f150lp_jec401nx_flt.fits,319.600926,0.292154,-0.00000433199,0.00000699853,0.00000830044,0.00000456465
hst_16231_03_acs_sbc_f150lp_jec403ib_flt.fits,319.600695,0.291350,-0.00000517460,0.00000648425,0.00000780337,0.00000527014
hst_16231_03_acs_sbc_f150lp_jec403ic_flt.fits,319.600782,0.291116,-0.00000517453,0.00000648416,0.00000780327,0.00000527007
hst_16231_03_acs_sbc_f150lp_jec403id_flt.fits,319.600627,0.291132,-0.00000517444,0.00000648405,0.00000780313,0.00000526998
hst_16231_03_acs_sbc_f150lp_jec403ie_flt.fits,319.600549,0.291367,-0.00000517438,0.00000648397,0.00000780304,0.00000526992


In [32]:
collect = ImageFileCollection('./OrIg_files/', filenames = lst_1 + lst_3, ext=2, glob_include = '*flt.fits',
                                 keywords=["CRVAL1", "CRVAL2", "CD1_1", "CD1_2",
                                           "CD2_1", "CD2_2"])
table = collect.summary
table['CRVAL1'].format = '.6f'
table['CRVAL2'].format = '.6f'
table['CD1_1'].format = '.11f'
table['CD1_2'].format = '.11f'
table['CD2_1'].format = '.11f'
table['CD2_2'].format = '.11f'
table

file,CRVAL1,CRVAL2,CD1_1,CD1_2,CD2_1,CD2_2
str45,float64,float64,float64,float64,float64,float64
hst_16231_01_acs_sbc_f150lp_jec401no_flt.fits,319.601084,0.292113,-0.00000433217,0.00000699882,0.00000830078,0.00000456484
hst_16231_01_acs_sbc_f150lp_jec401nr_flt.fits,319.601139,0.291870,-0.00000433212,0.00000699873,0.00000830067,0.00000456478
hst_16231_01_acs_sbc_f150lp_jec401nv_flt.fits,319.600980,0.291906,-0.00000433204,0.00000699861,0.00000830054,0.00000456470
hst_16231_01_acs_sbc_f150lp_jec401nx_flt.fits,319.600925,0.292148,-0.00000433199,0.00000699853,0.00000830044,0.00000456465
hst_16231_03_acs_sbc_f150lp_jec403ib_flt.fits,319.600705,0.291369,-0.00000517460,0.00000648425,0.00000780337,0.00000527014
hst_16231_03_acs_sbc_f150lp_jec403ic_flt.fits,319.600785,0.291134,-0.00000517453,0.00000648416,0.00000780327,0.00000527007
hst_16231_03_acs_sbc_f150lp_jec403id_flt.fits,319.600623,0.291153,-0.00000517444,0.00000648405,0.00000780313,0.00000526998
hst_16231_03_acs_sbc_f150lp_jec403ie_flt.fits,319.600543,0.291388,-0.00000517438,0.00000648397,0.00000780304,0.00000526992
